In [3]:
!pip install vtk


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import vtk
import numpy as np

In [2]:
import vtk
import numpy as np

def interpolate_vector(field_data, point):
    """
    Interpolate vector at a given point using VTKProbeFilter.
    """
    points = vtk.vtkPoints()
    points.InsertNextPoint(point[0],point[1],point[2])
    pointsPolydata = vtk.vtkPolyData()
    pointsPolydata.SetPoints(points)

    probe = vtk.vtkProbeFilter()
    probe.SetInputData(pointsPolydata)
    probe.SetSourceData(field_data)
    probe.Update()

    interpolated_data = probe.GetOutput()

    #num_arrays = interpolated_data.GetPointData().GetNumberOfArrays()
    #print("Number of arrays in interpolated data:", num_arrays)
    #for i in range(num_arrays):
    #    array_name = interpolated_data.GetPointData().GetArrayName(i)
    #    print("Array name:", array_name)

    interpolated_vector = interpolated_data.GetPointData().GetArray("vectors")
    vector = interpolated_vector.GetTuple(0)  # Assuming only one point is probed

    return vector

def rk4_integrate(field_data, seed, step_size, max_steps):
    """
    Perform RK4 integration to generate a streamline.
    """
    streamline_points = [seed]

    for _ in range(max_steps):
        current_point = streamline_points[-1]
        k1 = step_size*np.array(interpolate_vector(field_data, current_point))

        k2_point = current_point + 0.5 * k1
        k2 = step_size*np.array(interpolate_vector(field_data, k2_point))

        k3_point = current_point + 0.5 * k2
        k3 = step_size*np.array(interpolate_vector(field_data, k3_point))

        k4_point = current_point +  k3
        k4 = step_size * np.array(interpolate_vector(field_data, k4_point))

        next_point = current_point + (k1 + 2*k2 + 2*k3 + k4)/6.0

        # Check if next point is within bounds
        bounds = field_data.GetBounds()
        if (bounds[0] <= next_point[0] <= bounds[1] and
            bounds[2] <= next_point[1] <= bounds[3] and
            bounds[4] <= next_point[2] <= bounds[5]):
            streamline_points.append(next_point)
        else:
            break

    return np.array(streamline_points)

def create_vtp_file(streamline_points, filename):
    """
    Create a VTKPolyData (.vtp) file from the streamline points.
    """
    points = vtk.vtkPoints()
    lines = vtk.vtkCellArray()

    for point in streamline_points:
        points.InsertNextPoint(point)

    for i in range(len(streamline_points) - 1):
        line = vtk.vtkLine()
        line.GetPointIds().SetId(0, i)
        line.GetPointIds().SetId(1, i + 1)
        lines.InsertNextCell(line)

    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)
    polydata.SetLines(lines)

    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(filename)
    writer.SetInputData(polydata)
    writer.Write()

if __name__ == "__main__":
    # Load vector field data
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName("tornado3d_vector.vti")  # Replace with your file path
    reader.Update()
    field_data = reader.GetOutput()

    # Get seed location from user
    seed_x = float(input("Enter x coordinate of seed location: "))
    seed_y = float(input("Enter y coordinate of seed location: "))
    seed_z = float(input("Enter z coordinate of seed location: "))
    seed = np.array([seed_x, seed_y, seed_z])

    # Parameters
    step_size = 0.05
    max_steps = 1000

    # Perform RK4 integration
    streamline_forward = rk4_integrate(field_data, seed, step_size, max_steps)
    streamline_backward = rk4_integrate(field_data, seed, -step_size, max_steps)[::-1]  # Reverse the backward streamline

    # Combine forward and backward streamlines
    streamline_points = np.concatenate((streamline_backward[:-1], streamline_forward))
    print(len(streamline_points))
    
    # Create VTP file
    create_vtp_file(streamline_points, "main.vtp")
    print("Streamline file created successfully.")


Enter x coordinate of seed location:  0
Enter y coordinate of seed location:  0
Enter z coordinate of seed location:  7


638
Streamline file created successfully.
